In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI(model="gpt-4o") # AIMessage
    | StrOutputParser()
)

chain.invoke({"question": "who is james?"})

'Other'

In [5]:
langchain_chain = PromptTemplate.from_template(
    """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-4o")

anthropic_chain = PromptTemplate.from_template(
    """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-4o")

general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-4o")

In [6]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [ ]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)

In [8]:
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content="As Dario Amodei told me, using Anthropic involves understanding and engaging with advanced AI systems thoughtfully and safely. If you're interested in utilizing Anthropic's technologies or services, you would typically need to start by identifying the specific AI applications or research areas you are interested in. From there, you might consider reaching out through official channels, such as their website or professional networking platforms, to learn more about potential collaborations, tools, or resources that Anthropic offers. Ensuring you have a good grounding in AI principles and safety considerations is also key to effectively using what Anthropic develops.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 47, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {

In [9]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch


In [11]:
full_chain.invoke({"question": "who is linus tolvard ?"})

AIMessage(content='Linus Torvalds is a Finnish-American software engineer best known for creating the Linux kernel, which became the foundation for the Linux operating system family. Born on December 28, 1969, in Helsinki, Finland, Torvalds began developing Linux as a personal project in 1991 while attending the University of Helsinki. Linux has since grown into one of the most important open-source operating systems in the world, powering everything from servers and supercomputers to smartphones and embedded devices. Linus Torvalds has also been a key figure in the open-source movement and continues to oversee the development of the Linux kernel.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 24, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0